<a href="https://colab.research.google.com/github/Priyo-prog/Statistics-and-Data-Science/blob/main/Feature%20Selection%20Complete/Wrapper%20Method/wrapper_step_forward.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Step forward feature selection**
Step forward feature selection starts by training a machine learning model for each feature in the dataset and selecting, as the starting feature, the one that returns the best performing model according to the evaluation criteria we choose.

In the second step, it creates machine learning models for all combinations of the feature selected in the previous step and a second feature. It selects the pair that produces the best performing algorithm.

It continues by adding 1 feature at a time to the features that were selected in previous steps, until a stopping criteria is reached.

In theory, models with more features perform better. The algorithm will continue adding new features until a certain criteria is met. For example, until the model performance does not increase beyond a certain threshold. Or, as we show in this notebook, until a certain number of features are selected.

The model performance metric can be the roc_auc for classification and the r squared for regression, for example, and it is determined by the user.

Step forward feature selection is called a greedy procedure because it evaluates many possible single, double, triple, and so on feature combinations. Therefore, it is very computationally expensive and, sometimes, if the feature space is big enough, even unfeasible.

There is a special package in Python that implements this type of feature selection: mlxtend.

In the mlxtend implementation of the Step Forward Feature Selection, the original stopping criteria is an arbitrarily set number of features. So the search will finish when we reach the desired number of selected features.

This is somewhat arbitrary, we might be selecting a sub-optimal number of features, or likewise, a high number of features. But, by looking at the performance metric returned by the algorithm as it selects the features, we can have a view on whether more features do add value, or not.

**Note**

In latest releases, MLXtend incorporated alternative stopping criterias. Visit the documentation for more details.

Here I will use the Step Forward feature selection algorithm from MLXtend in a classification and regression dataset.

In [3]:
# Import important libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, r2_score

from sklearn.feature_selection import SequentialFeatureSelector